In [19]:
from dataClasses import *
from cachetools import cached
import copy
import time
import random
from typing import List, Set, Tuple, Dict
import pandas as pd
#import llist
import logging
import pprint
from vrpy import VehicleRoutingProblem
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from util import *
from validator.InstanceCO22 import InstanceCO22
import warnings
from collections import OrderedDict
from searchAlgorithms import randomLocalSearch
warnings.filterwarnings("ignore", module="matplotlib\..*")
warnings.filterwarnings("ignore", module="vrpy\..*")


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.CRITICAL)
%reload_ext autoreload
%autoreload 2


## Initial solution

In [20]:
def distance(loc1: InstanceCO22.Location, loc2: InstanceCO22.Location, ceil: bool = True) -> float:
    dist = math.sqrt((loc1.X - loc2.X)**2 + (loc1.Y - loc2.Y)**2)
    if ceil:
        return math.ceil(dist)
    else:
        return dist


def requestClosestHub(instance: InstanceCO22, request: InstanceCO22.Request) -> int:
    nHubs = len(instance.Hubs)
    hubs = instance.Locations[1:nHubs+1]
    minDist = math.inf
    minDistHubLocID = None
    for i, hub in enumerate(hubs):
        if request.ID in instance.Hubs[i].allowedRequests:
            hubDist = distance(
                instance.Locations[request.customerLocID-1], hub)
            if hubDist < minDist:
                minDist = hubDist
                minDistHubLocID = hub.ID
    return minDistHubLocID


def requestsClosestHub(instance: InstanceCO22) -> dict:
    # return dictionary of {'LOC_ID': ' NEAREST LOC_ID'}
    res = {}
    for req in instance.Requests:
        res[req.ID] = requestClosestHub(instance, req)
    return res


def requestsPerHub(instance: InstanceCO22) -> dict:
    closestHubPerRequest = requestsClosestHub(instance=instance)
    nHubs = len(instance.Hubs)
    hubLocIDs = list(range(2, nHubs+2))
    res = {val: [] for val in hubLocIDs}
    for hubLocID in hubLocIDs:
        for reqID, closestHubLocID in closestHubPerRequest.items():
            if closestHubLocID is hubLocID:
                res[hubLocID].append(reqID)
    return res


def amountPerProduct(instance: InstanceCO22, requests: list) -> list:
    nProducts = len(instance.Products)
    res = [None]*nProducts
    for i in range(nProducts):
        res[i] = sum([req.amounts[i] for req in requests])
    return res


def filterRequests(instance: InstanceCO22, day: int = None, locationsID: int = None) -> list:
    res = instance.Requests.copy()
    if day != None:
        res = [_ for _ in res if _.desiredDay is day]
    if locationsID != None:
        res = [_ for _ in res if _.customerLocID in locationsID]
    return res


def addAllEdges(G: nx.DiGraph, checkWindowOverlap: bool = False) -> nx.DiGraph:
    for locID1, node1 in G.nodes(data=True):
        for locID2, node2 in G.nodes(data=True):
            if locID1 != locID2:
                if checkWindowOverlap and locID1 not in ["Source", "Sink"] and locID2 not in ["Source", "Sink"] and not windowOverlap(node1['periodIDs'], node2['periodIDs']):
                    continue
                dist = math.ceil(
                    math.sqrt(pow(node1['X']-node2['X'], 2) + pow(node1['Y']-node2['Y'], 2)))
                if locID1 == "Sink" or locID2 == "Source" or (locID1 == "Source" and locID2 == "Sink"):
                    continue
                else:
                    G.add_edge(locID1, locID2, time=dist, cost=dist)
    return G


def createNxHub(instance: InstanceCO22, hubLocID: int, requests: list) -> nx.DiGraph:
    G = nx.DiGraph()
    for req in requests:
        reqLoc = instance.Locations[req.customerLocID-1]
        G.add_node(req.ID, locID=reqLoc.ID, reqID=req.ID, X=reqLoc.X,
                   Y=reqLoc.Y, demand=sum(req.amounts), amounts=req.amounts)
    hubLoc = instance.Locations[hubLocID]
    G.add_node("Source", locID=hubLocID, X=hubLoc.X, Y=hubLoc.Y)
    G.add_node("Sink", locID=hubLocID, X=hubLoc.X, Y=hubLoc.Y)
    G = addAllEdges(G)
    return G


def solveHubVRP(instance: InstanceCO22, hubLocID: int, requests: list) -> dict:
    # create networkX
    G = createNxHub(instance, hubLocID, requests)
    G_dict = {i: v for i, v in G.nodes(data=True)}
    # print(G_dict.keys())
    prob = VehicleRoutingProblem(G, load_capacity=instance.VanCapacity)
    prob.duration = instance.VanMaxDistance
    prob.fixed_cost = instance.VanDayCost
    prob.solve()
    best_routes = prob.best_routes
    #best_routes = {id:listReplace(best_routes[id], ["Source","Sink"], hubLocID) for id in best_routes.keys()}
    res = {
        'routes': {key: {'route': [G_dict[id] for id in best_routes[key]]} for key in best_routes.keys()},
        'demand': sum([sum(req.amounts) for req in requests]),
        'amounts': amountPerProduct(instance, requests)
    }
    return res


def createNxDepot(instance: InstanceCO22, dayRoutes: dict) -> nx.DiGraph:
    G = nx.DiGraph()
    # add hubs
    for (day, hubLocID), hubData in dayRoutes.items():
        G.add_node(f"{hubLocID}.1", locID=hubLocID, demand=0, amounts=0,
                   X=instance.Locations[hubLocID-1].X, Y=instance.Locations[hubLocID-1].Y)

    for (day, hubLocID), hubData in dayRoutes.items():
        i = 1
        nodeID = f"{hubLocID}.{i}"
        while nodeID in G.nodes and G.nodes[nodeID]['demand'] + hubData['demand'] > instance.TruckCapacity:
            i += 1
            nodeID = f"{hubLocID}.{i}"
        if nodeID not in G.nodes:
            G.add_node(nodeID, locID=hubLocID, demand=0, amounts=0,
                       X=instance.Locations[hubLocID-1].X, Y=instance.Locations[hubLocID-1].Y)

        G.nodes[nodeID]['amounts'] = list(
            np.array(G.nodes[nodeID]['amounts']) + np.array(hubData['amounts']))
        G.nodes[nodeID]['demand'] = G.nodes[nodeID]['demand']+hubData['demand']

    G.add_node("Source", locID=1,
               X=instance.Locations[0].X, Y=instance.Locations[0].Y)
    G.add_node("Sink", locID=1,
               X=instance.Locations[0].X, Y=instance.Locations[0].Y)

    G = addAllEdges(G)
    return G


def solveDepotVRP(instance: InstanceCO22, dayRoutes: dict) -> dict:
    # for solving per day
    G = createNxDepot(instance, dayRoutes)
    G_dict = {i: v for i, v in G.nodes(data=True)}
    print(G_dict.keys())

    prob = VehicleRoutingProblem(G, load_capacity=instance.TruckCapacity)
    prob.duration = instance.TruckMaxDistance
    prob.fixed_cost = instance.TruckDayCost
    prob.solve()
    best_routes = prob.best_routes
    # print(best_routes)
    #best_routes = {id:listReplace(best_routes[id], ["Source","Sink"], hubLocID) for id in best_routes.keys()}
    res = {key: [G_dict[id] for id in best_routes[key]]
           for key in best_routes.keys()}
    # print(res)
    return res


@cached(
    cache={},
    key=lambda a: a.CACHE_ID
)
def solveHub(instance: InstanceCO22) -> dict:
    nDays = instance.Days
    nHubs = len(instance.Hubs)
    hubLocIDs = list(range(2, nHubs+2))
    hubClusters = requestsPerHub(instance)
    hubRoutes = {}
    for day in range(1, nDays+1):  # hub routing
        dayRoutes = {}
        for hubLocID in hubLocIDs:
            hubCluster = hubClusters[hubLocID]
            #requestsToServe = filterRequests(instance, day, hubCluster)
            requestsToServe = [
                _ for _ in instance.Requests if _.ID in hubCluster and _.desiredDay is day]
            #print(hubCluster, [_.ID for _ in requestsToServe])   #
            if(len(requestsToServe) > 0):
                dayHubRoutes = solveHubVRP(instance, hubLocID, requestsToServe)
                dayRoutes[hubLocID] = dayHubRoutes
        hubRoutes[day] = dayRoutes
    return hubRoutes


In [21]:
def parseToDayHubroutes(res):
    i = 0
    hubRoutesObject = HubRoutes()
    # take dictionary with structure as hubRoutes datamodel
    for day, dayData in res.items():
        dayHubRoutesObject = DayHubRoutes()
        for hubLocID, hubData in dayData.items():
            for routeID, routeData in hubData['routes'].items():
                route = HubRoute(i, hubLocID)
                for node in routeData['route'][1:-1]:  # ignore source and sink
                    nodeObject = Node(
                        reqID=node['reqID'], locID=node['locID'], amounts=node['amounts'], X=node['X'], Y=node['Y'])
                    route.addNode(nodeObject)
                dayHubRoutesObject.addRoute(route)
        hubRoutesObject.addDayHubRoutes(
            day=day, dayHubRoutes=dayHubRoutesObject)
    return hubRoutesObject


## Integrated Local Search 

In [22]:

def dayHubRoutesOperators(instance, state: HubRoutes, distanceMatrix, operator: int, useHubOpeningCost=False):
    # apply specified operator to each day and store new state if better

    for day, dayHubRoutes in state.hubRoutes.items():  # local search depot routes per day

        dayHubRouteCost = dayHubRoutes.computeCost(
            instance.VanDistanceCost, instance.VanDayCost, distanceMatrix, useHubOpeningCost  = useHubOpeningCost, instance = instance)
        # copy the DayHubRoutes only so not every cost for every day is re-evaluated every time
        neighBour = copy.deepcopy(dayHubRoutes)

        operators = [
            neighBour.randomMergeRoutes,
            neighBour.randomNodeInsertion,
            neighBour.randomSectionInsertion,
            neighBour.shuffleRoute
        ]

        operatorToApply = operators[operator]

        if len(dayHubRoutes) > 0:
            operatorToApply()

        neighBourCost = neighBour.computeCost(
            instance.VanDistanceCost, instance.VanDayCost, distanceMatrix, useHubOpeningCost  = useHubOpeningCost, instance = instance)

        if neighBourCost < dayHubRouteCost:
            if neighBour.isFeasible(instance.VanCapacity, instance.VanMaxDistance, distanceMatrix, verbose=False):
                #print("OLD\n", dayHubRoutes)
                # replace old DayHubRoutes in HubRoutes for neighbor
                state.hubRoutes[day] = neighBour
                #print("NEW\n", neighBour)

    newStateCost = state.computeCost(instance.VanDistanceCost, instance.VanDayCost,
                                     instance.VanCost, instance.deliverEarlyPenalty, distanceMatrix)

    return state, newStateCost


def moveDayOperator(instance, state: HubRoutes, stateCost, distanceMatrix, useHubOpeningCost):
    # do one random move day earlier and replace state if it is better
    neighBour = copy.deepcopy(state)
    neighBour.randomMoveNodeDayEarly()
    neighBourCost = neighBour.computeCost(
        instance.VanDistanceCost, instance.VanDayCost, instance.VanCost, instance.deliverEarlyPenalty, distanceMatrix, useHubOpeningCost, instance)
    if neighBourCost < stateCost:
        if neighBour.isFeasible(instance.VanCapacity, instance.VanMaxDistance, distanceMatrix, verbose=False):
            neighBourCost = neighBour.computeCost(
                instance.VanDistanceCost, instance.VanDayCost, instance.VanCost, instance.deliverEarlyPenalty, distanceMatrix, useHubOpeningCost, instance)
            stateCost = neighBourCost
            #print("New best: ", stateCost)
            state = neighBour
    return state, stateCost


def integratedHubRoutesSearch(instance, hubRoutes: HubRoutes, maxTimeSeconds, maxIterations, nStart=5):
    # each iteration:
    #   One operation to HubRoutes
    #   One operation to every DayHubRoutes in Hubroutes
    maxTimeSeconds = maxTimeSeconds/nStart
    maxIterations = maxIterations/nStart

    bestState = None
    bestStateCost = math.inf

    useHubOpeningCost = any([_.hubOpeningCost > 0 for _ in instance.Hubs])

    dm = DistanceMatrix(instance)
    costBegin = hubRoutes.computeCost(
        instance.VanDistanceCost, instance.VanDayCost, instance.VanCost, instance.deliverEarlyPenalty, dm, useHubOpeningCost, instance)

    for iteration in range(nStart):
        startTime = time.time()
        state = copy.deepcopy(hubRoutes)
        stateCost = costBegin.copy()
        print("-"*10, f'Run {iteration}', "-"*10)
        print(f"starting cost: {stateCost}")
        i = 0
        optimizeDeliverEarly = (instance.deliverEarlyPenalty > 0)

        while time.time() - startTime < maxTimeSeconds and i < maxIterations:

            print("-")
            print(f"{iteration},{i} before operator 0: {stateCost}")
            state, stateCost = dayHubRoutesOperators(
                instance, state, distanceMatrix=dm, operator=0, useHubOpeningCost=useHubOpeningCost)
            print(f"{iteration},{i} before operator 1: {stateCost}")
            state, stateCost = dayHubRoutesOperators(
                instance, state, distanceMatrix=dm, operator=1, useHubOpeningCost=useHubOpeningCost)
            print(f"{iteration},{i} before operator moveDay: {stateCost}")
            if optimizeDeliverEarly:
                state, stateCost = moveDayOperator(
                    instance, state, stateCost, distanceMatrix=dm, useHubOpeningCost=useHubOpeningCost)
            print(f"{iteration},{i} before operator 2: {stateCost}")
            state, stateCost = dayHubRoutesOperators(
                instance, state, distanceMatrix=dm, operator=2, useHubOpeningCost=useHubOpeningCost)
            print(f"{iteration},{i} before operator 3: {stateCost}")
            state, stateCost = dayHubRoutesOperators(
                instance, state, distanceMatrix=dm, operator=3, useHubOpeningCost=useHubOpeningCost)
            print(f"{iteration},{i} end: {stateCost}")
            i += 1

        if stateCost < bestStateCost:
            bestState = state
            bestStateCost = stateCost

    costEnd = bestState.computeCost(instance.VanDistanceCost, instance.VanDayCost,
                                    instance.VanCost, instance.deliverEarlyPenalty, dm, useHubOpeningCost=useHubOpeningCost, instance=instance)
    print(
        f"Finished searchHubRoutes resulting in cost decrease of {costBegin}-{costEnd}={costBegin-costEnd}")
    return bestState


In [ ]:
instance = loadInstance(22)
instance.CACHE_ID = 22
res = solveHub(instance)
dm = DistanceMatrix(instance)
test = parseToDayHubroutes(res)
initialCost = test.computeCost(instance.VanDistanceCost, instance.VanDayCost,
                               instance.VanCost, instance.deliverEarlyPenalty, dm)
print(initialCost)


In [ ]:
ls_result = test
print(ls_result.computeCost(instance.VanDistanceCost, instance.VanDayCost,
      instance.VanCost, instance.deliverEarlyPenalty, dm))


In [ ]:
res, _ = dayHubRoutesOperators(instance, ls_result, dm, operator=0)
res.computeCost(instance.VanDistanceCost, instance.VanDayCost,
                instance.VanCost, instance.deliverEarlyPenalty, dm)


In [ ]:
res, _ = moveDayOperator(instance, ls_result, initialCost, dm, True)
res.computeCost(instance.VanDistanceCost, instance.VanDayCost,
                instance.VanCost, instance.deliverEarlyPenalty, dm, True, instance)


In [ ]:
res = integratedHubRoutesSearch(
    instance, hubRoutes=test, maxTimeSeconds=5, maxIterations=3000, nStart=100)
res.computeCost(instance.VanDistanceCost, instance.VanDayCost,
                instance.VanCost, instance.deliverEarlyPenalty, dm)


## Finishing using greedy02

In [23]:
def extractDays(hubRoutes, days: list) -> dict:
    res = {}
    for day, dayRoutes in hubRoutes.items():
        if day in days:
            for hubLocID in dayRoutes:
                newID = (day, hubLocID)
                res[newID] = dayRoutes[hubLocID]
    return res

def solveDepot(instance: InstanceCO22, hubRoutes, useDMin = True) -> dict:
    nDays = instance.Days
    nHubs = len(instance.Hubs)
    hubRoutesDict = hubRoutes.toDict(instance)

    depotRoutes = {}
    
    if useDMin:
        dmin = min([_.daysFresh for _ in instance.Products])
    else: 
        dmin = 1

    for i in range(0, math.ceil(nDays/dmin)):
        periodBegin = dmin*i+1
        periodEnd = dmin*(i+1)+1
        print(i, periodBegin, periodEnd)
        period = list(range(periodBegin, periodEnd))
        periodRoutes = extractDays(hubRoutesDict, period)
        if len(periodRoutes) > 0:  # depot routing
            res = solveDepotVRP(instance, periodRoutes)
            depotRoutes[periodBegin] = res
        else:
            depotRoutes[periodBegin] = {}

    return {'hubRoutes': hubRoutesDict, 'depotRoutes': depotRoutes}

def solve(instance: InstanceCO22, useDMin = True) -> dict:
    nDays = instance.Days
    nHubs = len(instance.Hubs)
    hubRoutes = solveHub(instance)
    hubRoutes = parseToDayHubroutes(hubRoutes)

    hubRoutes = integratedHubRoutesSearch(instance, hubRoutes, maxTimeSeconds=math.inf, maxIterations=5000, nStart=5)
    hubRoutesDict = hubRoutes.toDict(instance)

    depotRoutes = {}
    
    if useDMin:
        dmin = min([_.daysFresh for _ in instance.Products])
    else: 
        dmin = 1

    for i in range(0, math.ceil(nDays/dmin)):
        periodBegin = dmin*i+1
        periodEnd = dmin*(i+1)+1
        print(i, periodBegin, periodEnd)
        period = list(range(periodBegin, periodEnd))
        periodRoutes = extractDays(hubRoutesDict, period)
        if len(periodRoutes) > 0:  # depot routing
            res = solveDepotVRP(instance, periodRoutes)
            depotRoutes[periodBegin] = res
        else:
            depotRoutes[periodBegin] = {}

    return {'hubRoutes': hubRoutesDict, 'depotRoutes': depotRoutes}


In [24]:
def solutionToStr(instance: InstanceCO22, res: dict):
    resultString = "DATASET = CO2022_11 \n \n"

    for day in range(1, instance.Days+1):
        resultString += f"DAY = {day} \n"

        truckString = ""
        if day in res['depotRoutes'].keys():
            nTrucks = len(res['depotRoutes'][day])
            for routeID, truckRoute in res['depotRoutes'][day].items():
                truckString += f"{routeID} "
                for i, hubData in enumerate(truckRoute[1:-1]):
                    amountPerProduct = hubData['amounts']
                    truckString += f"H{hubData['locID'] - 1} {','.join([str(_) for _ in amountPerProduct])} "
                truckString += "\n"
        else:
            nTrucks = 0

        resultString += f"NUMBER_OF_TRUCKS = {nTrucks} \n"
        resultString += truckString

        nVans = 0
        i = 0
        vanString = ""
        for hubLocID in res['hubRoutes'][day].keys():
            for _, route in res['hubRoutes'][day][hubLocID]['routes'].items():
                i += 1
                reqIds = [_['reqID'] for _ in route['route'][1:-1]]
                vanString += f"{i} H{hubLocID-1} {' '.join([str(_) for _ in reqIds])} \n"
            nVans += len(res['hubRoutes'][day][hubLocID]['routes'])
        resultString += f"NUMBER_OF_VANS = {nVans} \n"
        resultString += vanString + "\n"
    return resultString


In [ ]:
i = 26
instance = loadInstance(i)
instance.CACHE_ID = i
res = solve(instance, useDMin = False)
solutionStr = solutionToStr(instance, res)
with open(f"./solutions/greedy01ls/solution{i}_DE.txt", 'w') as file:
    file.write(solutionStr)


In [ ]:
for i in range(1,31):
    print(i)
    instance = loadInstance(i)
    instance.CACHE_ID = i
    useDMin = True if i not in [6,16] else False
    res = solve(instance, useDMin)
    solutionStr = solutionToStr(instance, res)
    with open(f"./solutions/greedy01ls/solution{i}_DE.txt", 'w') as file:
        file.write(solutionStr)

In [25]:
for i in [27]:
    instance = loadInstance(i)
    initialStateGenerator = lambda instance: parseToDayHubroutes(solveHub(instance))
    searcher = randomLocalSearch(instance, 500, math.inf, 1, None, initialStateGenerator)
    bestState, costs = searcher.run(verbose=True)
    res = solveDepot(instance, searcher.bestState)
    solutionStr = solutionToStr(instance, res)

    #with open(f"./solutions/integratedls/solution{i}_DE2.txt", 'w') as file:
        #file.write(solutionStr)


INFO:vrpy.vrp:new upper bound : max num stops = 3
INFO:vrpy.vrp:Clarke & Wright solution found with value 506 and 1 vehicles
INFO:vrpy.vrp:Greedy solution found with value 506 and 1 vehicles
INFO:vrpy.vrp:iteration 0, 506.0
INFO:vrpy.master_solve_pulp:total cost = 506.0
INFO:vrpy.vrp:new upper bound : max num stops = 3
INFO:vrpy.vrp:Clarke & Wright solution found with value 536 and 1 vehicles
INFO:vrpy.vrp:Greedy solution found with value 536 and 1 vehicles
INFO:vrpy.vrp:iteration 0, 536.0
INFO:vrpy.master_solve_pulp:total cost = 536.0
INFO:vrpy.vrp:new upper bound : max num stops = 3
INFO:vrpy.vrp:Clarke & Wright solution found with value 474 and 1 vehicles
INFO:vrpy.vrp:Greedy solution found with value 474 and 1 vehicles
INFO:vrpy.vrp:iteration 0, 474.0
INFO:vrpy.master_solve_pulp:total cost = 474.0
INFO:vrpy.vrp:new upper bound : max num stops = 3
INFO:vrpy.vrp:Clarke & Wright solution found with value 512 and 1 vehicles
INFO:vrpy.vrp:Greedy solution found with value 512 and 1 vehic

Generated initial state
useCheckHubCanServe True
-------------------- Run 0 --------------------
initialCost: 176655.0
Progress: [------------------> ] 99%  139060.0 [379] NB better than current state: 139060.0 DayHubRoutes operator 3mMoveNodeDayEarly
 End of iteration, stateCost obtained: 139060.0
Current better than best state: 139060.0
0 1 5
dict_keys(['10.1', '20.1', '15.1', '8.1', '19.1', '6.1', '4.1', '13.1', '7.1', 'Source', 'Sink'])


INFO:vrpy.vrp:new upper bound : max num stops = 11
INFO:vrpy.vrp:Clarke & Wright solution found with value 2161 and 1 vehicles
INFO:vrpy.vrp:Greedy solution found with value 2197 and 1 vehicles
INFO:vrpy.vrp:iteration 0, 2161.0
INFO:vrpy.vrp:iteration 1, 2161.0
INFO:vrpy.vrp:iteration 2, 2161.0
INFO:vrpy.vrp:iteration 3, 2161.0
INFO:vrpy.vrp:iteration 4, 2161.0
INFO:vrpy.vrp:iteration 5, 2161.0
INFO:vrpy.vrp:iteration 6, 2161.0
INFO:vrpy.vrp:iteration 7, 2161.0
INFO:vrpy.vrp:iteration 8, 2161.0
INFO:vrpy.vrp:iteration 9, 2161.0
INFO:vrpy.vrp:iteration 10, 2161.0
INFO:vrpy.vrp:iteration 11, 2161.0
INFO:vrpy.vrp:iteration 12, 2161.0
INFO:vrpy.vrp:iteration 13, 2161.0
INFO:vrpy.vrp:iteration 14, 2161.0
INFO:vrpy.vrp:iteration 15, 2161.0
INFO:vrpy.vrp:iteration 16, 2161.0
INFO:vrpy.vrp:iteration 17, 2161.0
INFO:vrpy.vrp:iteration 18, 2161.0
INFO:vrpy.master_solve_pulp:total cost = 2161.0
INFO:vrpy.vrp:new upper bound : max num stops = 9
INFO:vrpy.vrp:Clarke & Wright solution found with val

1 5 9
dict_keys(['19.1', '14.1', '15.1', '6.1', '12.1', '20.1', '10.1', 'Source', 'Sink'])


INFO:vrpy.vrp:iteration 2, 2209.0
INFO:vrpy.vrp:iteration 3, 2209.0
INFO:vrpy.vrp:iteration 4, 2209.0
INFO:vrpy.vrp:iteration 5, 2209.0
INFO:vrpy.vrp:iteration 6, 2209.0
INFO:vrpy.vrp:iteration 7, 2209.0
INFO:vrpy.vrp:iteration 8, 2209.0
INFO:vrpy.vrp:iteration 9, 2209.0
INFO:vrpy.vrp:iteration 10, 2209.0
INFO:vrpy.vrp:iteration 11, 2209.0
INFO:vrpy.vrp:iteration 12, 2209.0
INFO:vrpy.vrp:iteration 13, 2209.0
INFO:vrpy.vrp:iteration 14, 2209.0
INFO:vrpy.vrp:iteration 15, 2209.0
INFO:vrpy.master_solve_pulp:total cost = 2209.0
INFO:vrpy.vrp:new upper bound : max num stops = 10
INFO:vrpy.vrp:Clarke & Wright solution found with value 2215 and 1 vehicles
INFO:vrpy.vrp:Greedy solution found with value 2258 and 1 vehicles
INFO:vrpy.vrp:iteration 0, 2215.0
INFO:vrpy.vrp:iteration 1, 2215.0


...

INFO:vrpy.vrp:iteration 2, 2215.0
INFO:vrpy.vrp:iteration 3, 2215.0
INFO:vrpy.vrp:iteration 4, 2215.0
INFO:vrpy.vrp:iteration 5, 2215.0
INFO:vrpy.vrp:iteration 6, 2215.0
INFO:vrpy.vrp:iteration 7, 2215.0
INFO:vrpy.vrp:iteration 8, 2215.0
INFO:vrpy.vrp:iteration 9, 2215.0
INFO:vrpy.vrp:iteration 10, 2215.0
INFO:vrpy.vrp:iteration 11, 2215.0
INFO:vrpy.vrp:iteration 12, 2215.0
INFO:vrpy.vrp:iteration 13, 2215.0
INFO:vrpy.vrp:iteration 14, 2215.0
INFO:vrpy.vrp:iteration 15, 2215.0
INFO:vrpy.vrp:iteration 16, 2215.0
INFO:vrpy.vrp:iteration 17, 2215.0
INFO:vrpy.vrp:iteration 18, 2215.0
INFO:vrpy.vrp:iteration 19, 2215.0
INFO:vrpy.vrp:iteration 20, 2215.0
INFO:vrpy.vrp:iteration 21, 2215.0
INFO:vrpy.vrp:iteration 22, 2215.0
INFO:vrpy.vrp:iteration 23, 2215.0
INFO:vrpy.vrp:iteration 24, 2215.0
INFO:vrpy.vrp:iteration 25, 2215.0
INFO:vrpy.vrp:iteration 26, 2215.0
INFO:vrpy.vrp:iteration 27, 2215.0
INFO:vrpy.vrp:iteration 28, 2215.0
INFO:vrpy.vrp:iteration 29, 2215.0
INFO:vrpy.vrp:iteration 30, 

3 13 17
dict_keys(['12.1', '11.1', '4.1', '13.1', '6.1', '18.1', '2.1', '19.1', 'Source', 'Sink'])


INFO:vrpy.vrp:iteration 4, 2271.0
INFO:vrpy.vrp:iteration 5, 2271.0
INFO:vrpy.vrp:iteration 6, 2271.0
INFO:vrpy.vrp:iteration 7, 2271.0
INFO:vrpy.vrp:iteration 8, 2271.0
INFO:vrpy.vrp:iteration 9, 2271.0
INFO:vrpy.vrp:iteration 10, 2271.0
INFO:vrpy.vrp:iteration 11, 2271.0
INFO:vrpy.vrp:iteration 12, 2271.0
INFO:vrpy.vrp:iteration 13, 2271.0
INFO:vrpy.vrp:iteration 14, 2271.0
INFO:vrpy.vrp:iteration 15, 2271.0
INFO:vrpy.vrp:iteration 16, 2271.0
INFO:vrpy.vrp:iteration 17, 2271.0
INFO:vrpy.vrp:iteration 18, 2271.0
INFO:vrpy.vrp:iteration 19, 2271.0
INFO:vrpy.vrp:iteration 20, 2271.0
INFO:vrpy.vrp:iteration 21, 2271.0
INFO:vrpy.vrp:iteration 22, 2264.0
INFO:vrpy.vrp:iteration 23, 2264.0
INFO:vrpy.vrp:iteration 24, 2264.0
INFO:vrpy.vrp:iteration 25, 2264.0
INFO:vrpy.vrp:iteration 26, 2264.0
INFO:vrpy.vrp:iteration 27, 2264.0
INFO:vrpy.master_solve_pulp:total cost = 2264.0
INFO:vrpy.vrp:new upper bound : max num stops = 10
INFO:vrpy.vrp:Clarke & Wright solution found with value 2238 and 1 

4 17 21
dict_keys(['18.1', '10.1', '7.1', '13.1', '14.1', '8.1', '2.1', '15.1', 'Source', 'Sink'])


INFO:vrpy.vrp:iteration 3, 2238.0
INFO:vrpy.vrp:iteration 4, 2238.0
INFO:vrpy.vrp:iteration 5, 2238.0
INFO:vrpy.vrp:iteration 6, 2238.0
INFO:vrpy.vrp:iteration 7, 2238.0
INFO:vrpy.vrp:iteration 8, 2238.0
INFO:vrpy.vrp:iteration 9, 2238.0
INFO:vrpy.vrp:iteration 10, 2238.0
INFO:vrpy.vrp:iteration 11, 2238.0
INFO:vrpy.vrp:iteration 12, 2238.0
INFO:vrpy.vrp:iteration 13, 2238.0
INFO:vrpy.vrp:iteration 14, 2238.0
INFO:vrpy.vrp:iteration 15, 2238.0
INFO:vrpy.vrp:iteration 16, 2238.0
INFO:vrpy.vrp:iteration 17, 2238.0
INFO:vrpy.vrp:iteration 18, 2238.0
INFO:vrpy.vrp:iteration 19, 2238.0
INFO:vrpy.vrp:iteration 20, 2238.0
INFO:vrpy.vrp:iteration 21, 2238.0
INFO:vrpy.vrp:iteration 22, 2238.0
INFO:vrpy.vrp:iteration 23, 2238.0
INFO:vrpy.vrp:iteration 24, 2238.0
INFO:vrpy.vrp:iteration 25, 2238.0
INFO:vrpy.vrp:iteration 26, 2238.0
INFO:vrpy.vrp:iteration 27, 2238.0
INFO:vrpy.master_solve_pulp:total cost = 2238.0


In [ ]:
plt.plot(searcher.costs)

In [ ]:
res = solveDepot(instance, searcher.bestState)
solutionStr = solutionToStr(instance, res)

with open(f"./solutions/greedy01ls/solution{i}_DE2.txt", 'w') as file:
    file.write(solutionStr)

In [ ]:
i = 26
instance = loadInstance(i)
instance.Hubs
dm = DistanceMatrix(instance)
reqID = 2
req = instance.Requests[reqID-1]

hubID = 20
hubLocID = hubID + 1
print(req.ID)
print(instance.Hubs[hubLocID-2].ID)
print(dm.reqToHubDist(req.ID,req.customerLocID, hubLocID))
print(req.ID in instance.Hubs[hubLocID-2].allowedRequests)

In [ ]:
i = 26
loadInstance(i).Hubs